In [37]:
# Install required libraries
!pip install llama-index
!pip install llama-index-retrievers-bm25
!pip install chromadb
!pip install PyStemmer


  Using cached llama_index_core-0.11.23-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.11.23-py3-none-any.whl (1.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.12.4
    Uninstalling llama-index-core-0.12.4:
      Successfully uninstalled llama-index-core-0.12.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-cohere 0.4.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-embeddings-huggingface 0.4.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-extractors-entity 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-readers-database 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-readers-deeplake 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-retrievers-bm25 0.5.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which i

  Using cached llama_index_core-0.12.4-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.12.4-py3-none-any.whl (1.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.11.23
    Uninstalling llama-index-core-0.11.23:
      Successfully uninstalled llama-index-core-0.11.23


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.11.23 requires llama-index-core<0.12.0,>=0.11.23, but you have llama-index-core 0.12.4 which is incompatible.
llama-index-agent-openai 0.3.4 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.4 which is incompatible.
llama-index-cli 0.3.1 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.4 which is incompatible.
llama-index-embeddings-ollama 0.3.1 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.4 which is incompatible.
llama-index-embeddings-openai 0.2.5 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.4 which is incompatible.
llama-index-indices-managed-llama-cloud 0.4.0 requires llama-index-core<0.12.0,>=0.11.13.post1, but you have llama-index-core 0.12.4 which is incompatible.
llama-in

In [38]:
# Import necessary libraries
import os
import logging
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.query_engine import RetrieverQueryEngine
import chromadb
import Stemmer

# Configure logging
logging.basicConfig(level=logging.INFO)

In [39]:
from llama_index.core import Settings


# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)


Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

In [40]:
# Load documents
documents = SimpleDirectoryReader(input_files=["../data/paul_graham_essay3.txt"]).load_data()

# Initialize a SentenceSplitter to create nodes
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=512)
nodes = splitter.get_nodes_from_documents(documents)


In [41]:
# Create a BM25 Retriever
bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=2,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

# Persist the BM25 Retriever to disk
bm25_retriever.persist("./bm25_retriever")

# Reload the BM25 Retriever
loaded_bm25_retriever = BM25Retriever.from_persist_dir("./bm25_retriever")


DEBUG:bm25s:Building index from IDs objects


Finding newlines for mmindex:   0%|          | 0.00/259k [00:00<?, ?B/s]

In [42]:
# Initialize a docstore to store nodes
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

# Configure Chroma vector store
import chromadb
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("dense_vectors")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Create a QueryFusionRetriever with BM25 and Chroma
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.retrievers.bm25 import BM25Retriever

# Create the BM25 retriever
bm25_retriever = BM25Retriever.from_defaults(docstore=docstore, similarity_top_k=2)

# Create the VectorStoreIndex and its retriever
index = VectorStoreIndex(nodes=nodes, docstore=docstore, vector_store=vector_store)
vector_retriever = index.as_retriever(similarity_top_k=2)

# Combine retrievers into a QueryFusionRetriever
hybrid_retriever = QueryFusionRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    num_queries=1,
    use_async=False,
)

# Query the hybrid retriever
query = "What happened at Viaweb and Interleaf?"
retrieved_nodes = hybrid_retriever.retrieve(query)

# Display the results
for node in retrieved_nodes:
    print(f"Node ID: {node.node_id}")
    print(f"Text: {node.text}\n")


DEBUG:bm25s:Building index from IDs objects


Node ID: 04e28e51-73ab-4e1c-bee2-98dcb3453093
Text: This name didn't last long before it was replaced by "software as a service," but it was current for long enough that I named this new company after it: it was going to be called Aspra.

I started working on the application builder, Dan worked on network infrastructure, and the two undergrads worked on the first two services (images and phone calls). But about halfway through the summer I realized I really didn't want to run a company — especially not a big one, which it was looking like this would have to be. I'd only started Viaweb because I needed the money. Now that I didn't need money anymore, why was I doing this? If this vision had to be realized as a company, then screw the vision. I'd build a subset that could be done as an open source project.

Much to my surprise, the time I spent working on this stuff was not wasted after all. After we started Y Combinator, I would often encounter startups working on parts of this new arch

In [43]:
# Query the hybrid retriever
retrieved_nodes = hybrid_retriever.retrieve("What happened at Viaweb and Interleaf?")

# Display retrieved nodes
for node in retrieved_nodes:
    print(f"Node ID: {node.node_id}")
    print(f"Text: {node.text}\n")


Node ID: 04e28e51-73ab-4e1c-bee2-98dcb3453093
Text: This name didn't last long before it was replaced by "software as a service," but it was current for long enough that I named this new company after it: it was going to be called Aspra.

I started working on the application builder, Dan worked on network infrastructure, and the two undergrads worked on the first two services (images and phone calls). But about halfway through the summer I realized I really didn't want to run a company — especially not a big one, which it was looking like this would have to be. I'd only started Viaweb because I needed the money. Now that I didn't need money anymore, why was I doing this? If this vision had to be realized as a company, then screw the vision. I'd build a subset that could be done as an open source project.

Much to my surprise, the time I spent working on this stuff was not wasted after all. After we started Y Combinator, I would often encounter startups working on parts of this new arch

In [44]:
# Save the docstore
docstore.persist("./docstore.json")

# Reload docstore
from llama_index.core.storage.docstore import SimpleDocumentStore
docstore = SimpleDocumentStore.from_persist_path("./docstore.json")

# Reload Chroma vector store
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("dense_vectors")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Reload nodes from the docstore
from llama_index.core import VectorStoreIndex
nodes = [docstore.docs[doc_id] for doc_id in docstore.docs.keys()]

# Recreate VectorStoreIndex using the reloaded docstore, vector store, and nodes
index = VectorStoreIndex(
    nodes=nodes,  # Ensure nodes are added to the index
    docstore=docstore,
    vector_store=vector_store,
)


In [45]:
# Access all documents in the docstore
doc_ids = list(docstore.docs.keys())
print(f"Number of documents in docstore: {len(doc_ids)}")

# Optionally, print the IDs of stored documents
print("Document IDs in docstore:")
for doc_id in doc_ids:
    print(doc_id)


Number of documents in docstore: 57
Document IDs in docstore:
f79f4cda-be4f-48ac-a6de-6bdf34b5fdd9
f8569915-c223-4851-97d9-b557341dd08e
4a4d1ca1-494d-4403-bffe-b579ad02fb33
243c465a-d394-4b73-a0a9-9adf330dad7a
431a9415-0a6f-4f16-9129-da8949fc674c
68dedf92-9f64-4e99-8682-fcf7f9285f5f
839e622d-f951-4283-9b6e-39597ad81659
66d3887f-16aa-49a2-ab91-f3638ebd6526
b94dd7da-707d-486f-8fb9-81421cd689d0
acd8de89-6779-41f3-8d22-e74341fc9e4b
2422a3f1-4a01-42e7-ad24-b867d397e57b
5e2d79dd-0849-4334-9323-92df2ca6f64d
ea3fc457-d12a-4ee3-aaa8-78171f73de5d
3d2515ce-bc0d-4362-84da-56aa90f820f2
b7b57b26-e7fa-422d-abb8-d4db099f610d
214501b3-7658-42e2-8bc0-ce220dfc48e9
27df1539-e428-42df-af67-3f1dfad97cec
2c45f442-c9d7-4dda-8fb7-b445e4909a72
1839c962-f327-4bca-bddd-bfab1cc565bf
64961232-d8ce-4ed7-9a78-e7f18ab683de
47c7af94-2221-4ea0-8d37-978a36a3060f
f38f6e73-4382-4250-9b22-090e27549287
5333a6b8-3b2c-4e1b-b972-3ac69269ff5b
c0840f0d-0afc-4a24-a521-199080e9151f
e25b02a5-04fe-49d0-9646-730ef936a06f
4d074a72-e3dc

In [46]:
# Check if the vector store is operational
print(f"Vector store contains collection: {chroma_collection.name}")

# Optionally, check the number of entries in the collection
print(f"Number of vectors in vector store: {len(chroma_collection.get())}")


Vector store contains collection: dense_vectors
Number of vectors in vector store: 7


In [47]:
# Test query to ensure the index works as expected
query = "Who is the author of this essay?"
retriever = index.as_retriever(similarity_top_k=2)
retrieved_nodes = retriever.retrieve(query)

# Print the results
print("Retrieved Nodes:")
for node in retrieved_nodes:
    print(f"Node ID: {node.node_id}")
    print(f"Text: {node.text}\n")


Retrieved Nodes:
Node ID: c2c0daf3-ce47-4fc5-92fc-090f6e997946
Text: I still get the glassy eye from strangers when they ask what I'm writing, and I explain that it's an essay I'm going to publish on my web site. Even Lisp, though prestigious intellectually in something like the way Latin is, also seems about as hip.

It's not that unprestigious types of work are good per se. But when you find yourself drawn to some kind of work despite its current lack of prestige, it's a sign both that there's something real to be discovered there, and that you have the right kind of motives. Impure motives are a big danger for the ambitious. If anything is going to lead you astray, it will be the desire to impress people. So while working on things that aren't prestigious doesn't guarantee you're on the right track, it at least guarantees you're not on the most common type of wrong one.

Over the next several years I wrote lots of essays about all kinds of different topics. O'Reilly reprinted a coll